# Project 1: Digital Divide

#### Based on PPIC's Just the Facts report ["California's Digital Divide"](https://www.ppic.org/publication/californias-digital-divide/)

## Research Question(s):
1. What share households have access to high-speed internet? 
2. Does this number vary across demographic groups? (in this case race/ethnicity).

## Goal:
* explore datafiles (`acsdata.data.gz`) and create a _working dataset_ from it.

## Context:
Obtained American Community Survey (ACS) survey data from [IPUMS](https://usa.ipums.org/usa/). <br>
It contains basic demographics:
  - age
  - gender
  - race/ethnicity

and geographic indicators:
  - state
  - county

***

**Step 1: Set up your working environment.**

Import all necessary libraries and create `Path`s to your data directories. This ensures reproducibility across file systems (windows uses `\` instead of `/`)

In [1]:
# setting up working environment
import pandas as pd
from pathlib import Path
from datetime import datetime as dt
today = dt.today().strftime("%d-%b-%y")

print(today)

29-Apr-19


In [2]:
# data folder and paths
RAW_DATA_FOLDER = Path("../data/raw/")
INTERIM_DATA_FOLDER = Path("../data/interim/")
PROCESSED_DATA_FOLDER = Path("../data/processed/")
FINAL_DATA_FOLDER = Path("../data/final/")

In [3]:
from tools import tree
import gzip

In [4]:
tree(RAW_DATA_FOLDER)

+ ..\data\raw
    + acs_data.dta.gz
    + county_shapes.zip
    + cps_data.dta.gz
    + usa_00052.csv.gz
    + usa_00053.dta.gz


In [5]:
# loading data and exploratory analysis
with gzip.open(RAW_DATA_FOLDER / 'usa_00053.dta.gz') as datafile:
    data = pd.read_stata(datafile)

In [6]:
data.shape

(3190040, 18)

In [7]:
data.describe()

,serial,hhwt,countyfip,pernum,perwt
count,3.190040e+06,3.190040e+06,3.190040e+06,3.190040e+06,3.190040e+06
mean,6.918400e+05,9.614892e+01,5.099185e+01,2.083905e+00,1.021050e+02
std,4.064117e+05,7.594648e+01,8.809394e+01,1.340533e+00,8.314945e+01
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,3.350008e+05,5.100000e+01,0.000000e+00,1.000000e+00,5.300000e+01
50%,6.925320e+05,7.600000e+01,1.900000e+01,2.000000e+00,8.000000e+01
75%,1.047493e+06,1.170000e+02,7.300000e+01,3.000000e+00,1.240000e+02
max,1.394399e+06,2.401000e+03,8.100000e+02,2.000000e+01,2.401000e+03


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3190040 entries, 0 to 3190039
Data columns (total 18 columns):
year         category
serial       int32
hhwt         int16
statefip     category
countyfip    int16
gq           category
cinethh      category
cihispeed    category
pernum       int8
perwt        int16
relate       category
related      category
sex          category
age          category
race         category
raced        category
hispan       category
hispand      category
dtypes: category(13), int16(3), int32(1), int8(1)
memory usage: 100.4 MB


In [9]:
data.head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
0,2017,1,206,alabama,0,households under 1970 definition,"no internet access at this house, apartment, o...",n/a (gq),1,206,head/householder,head/householder,male,73,black/african american/negro,black/african american/negro,not hispanic,not hispanic
1,2017,2,45,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,45,head/householder,head/householder,female,31,white,white,not hispanic,not hispanic
2,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,1,136,head/householder,head/householder,male,41,white,white,mexican,mexican
3,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,2,121,spouse,spouse,female,48,white,white,not hispanic,not hispanic
4,2017,3,136,alabama,0,households under 1970 definition,"yes, with a subscription to an internet service",no,3,111,child,child,male,16,white,white,mexican,mexican


Subset california
```python
state_data = data[data['statefip'] == 'california'].copy()
```
or
```python
mask_state = data['statefip'] == 'california'
state_data = data[mask_state].copy()
```
or
```python
california_mask = data['statefip'] == 'california'
state_data = data.loc[mask_state, :].copy()
```


In [10]:
# subset california
mask_state = data['statefip'] == 'california'
state_data = data.loc[mask_state, ].copy()

In [11]:
state_data.head()

,year,serial,hhwt,statefip,countyfip,gq,cinethh,cihispeed,pernum,perwt,relate,related,sex,age,race,raced,hispan,hispand
153275,2017,68211,59,california,67,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,59,head/householder,head/householder,male,45,two major races,white and other asian race(s),not hispanic,not hispanic
153276,2017,68211,59,california,67,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,57,spouse,spouse,female,39,two major races,white and other asian race(s),not hispanic,not hispanic
153277,2017,68211,59,california,67,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,54,child,child,male,15,two major races,white and other asian race(s),not hispanic,not hispanic
153278,2017,68212,74,california,1,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,75,head/householder,head/householder,male,34,chinese,chinese,not hispanic,not hispanic
153279,2017,68212,74,california,1,households under 1970 definition,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,71,spouse,spouse,female,33,chinese,chinese,not hispanic,not hispanic


In [13]:
# analysis 1 - trim your data set
## drop unnecessary variables OR select necessary variables and create another subset
state_data.drop(columns = ['gq', 'related', 'raced', 'hispand',])

,year,serial,hhwt,stateicp,countyfip,cinethh,cihispeed,pernum,perwt,relate,sex,age,race,hispan
153275,2017,68211,59,california,67,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,59,head/householder,male,45,two major races,not hispanic
153276,2017,68211,59,california,67,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,57,spouse,female,39,two major races,not hispanic
153277,2017,68211,59,california,67,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,54,child,male,15,two major races,not hispanic
153278,2017,68212,74,california,1,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,75,head/householder,male,34,chinese,not hispanic
153279,2017,68212,74,california,1,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,71,spouse,female,33,chinese,not hispanic
153280,2017,68212,74,california,1,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,75,child,male,4,chinese,not hispanic
153281,2017,68212,74,california,1,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",4,75,child,male,1,chinese,not hispanic
153282,2017,68213,63,california,19,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,63,head/householder,female,76,white,not hispanic
153283,2017,68214,75,california,67,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,74,head/householder,male,35,other asian or pacific islander,not hispanic
153284,2017,68214,75,california,67,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,78,spouse,female,31,other asian or pacific islander,not hispanic


In [14]:
state_data['gq'].unique()

[households under 1970 definition, additional households under 1990 definition, additional households under 2000 definition, group quarters--institutions, other group quarters]
Categories (5, object): [households under 1970 definition < additional households under 1990 definition < group quarters--institutions < other group quarters < additional households under 2000 definition]

In [15]:
state_data['gq'].value_counts()

households under 1970 definition               359972
other group quarters                             8392
group quarters--institutions                     7575
additional households under 1990 definition      1503
additional households under 2000 definition       133
Name: gq, dtype: int64

From IPUMS docs:
>There are three slightly different definitions of group quarters in the IPUMS. For the period 1940-1970 (excluding the 1940 100% dataset), group quarters are housing units with five or more individuals unrelated to the householder. Before 1940 and in 1980-1990, units with 10 or more individuals unrelated to the householder are considered group quarters. **In the 2000 census, 2010 census, the ACS and the PRCS, no threshold was applied; for a household to be considered group quarters, it had to be on a list of group quarters that is continuously maintained by the Census Bureau. In earlier years, a similar list was used, with the unrelated-persons rule imposed as a safeguard.**

For this reason we will keep our analysis to households under the 1970 and 1990 definition.

conditional indexing - households under a certain household condition

In [16]:
# need to use parenthesis because of order of 
mask_household = (state_data['gq'] == 'households under 1970 definition') | (state_data['gq'] == 'additional households under 1990 definition')

Since `gq` is a categorical variable you could do the following
```python
household_mask = (california_data['gq'] <= 'households under 1990 definition')
```

In [17]:
print(state_data.shape)
state_data = state_data[mask_household].copy()
print(state_data.shape)

(377575, 18)
(361475, 18)


Let's drop those extra variables now.

In [18]:
state_data = state_data.drop(columns = ['gq', 'related', 'raced', 'hispand',])

indexing by another column: we want households with school-age children (18 or less) 

What we need
1. the `serial` value for those households where school-age children exist
2. a mask that checks if the `serial` value of a household in our `california_data` is in our `households_with_children` dataset.

The main challenge here is that the **unit of observation** in our `california_data` dataset is a person, whereas in our new `households_with_children` dataset the **unit of observation** will be a household.

In [19]:
# we use quotations because age is categorical right now. K-12 think 5 to 18
mask_children = (state_data['age'] <= '18') & (state_data['age'] >= '5')

In [20]:
households_with_children = state_data[mask_children].copy()
households_with_children.head()

,year,serial,hhwt,stateicp,countyfip,cinethh,cihispeed,pernum,perwt,relate,sex,age,race,hispan
153277,2017,68211,59,california,67,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,54,child,male,15,two major races,not hispanic
153292,2017,68217,31,california,29,"yes, with a subscription to an internet service",no,3,41,child,female,11,white,not hispanic
153293,2017,68217,31,california,29,"yes, with a subscription to an internet service",no,4,42,child,male,7,white,not hispanic
153296,2017,68218,103,california,55,"yes, with a subscription to an internet service",no,3,74,child,female,17,white,mexican
153320,2017,68229,50,california,37,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",4,50,grandchild,male,7,white,mexican


You'll notice that we have **duplicates** in our `serial` column. This is because it is possible to have more that one child living in a household. We're interested only in having a list of household `serial` values for those with children so we can ignore the rest of the columns. 

In [21]:
households_with_children = households_with_children['serial']

Now, households_with_children is a pandas `Series` (not a dataframe anymore)

In [22]:
households_with_children.head()

153277    68211
153292    68217
153293    68217
153296    68218
153320    68229
Name: serial, dtype: int32

A series has an index and a value.

In [23]:
households_with_children.shape

(63415,)

Notice there's no 2nd value (number of columns). Series are one-dimensional.

In [24]:
households_with_children.drop_duplicates().shape

(36808,)

let's see what we know so far.

In [25]:
state_data.shape

(361475, 14)

In [26]:
n_obs = state_data.shape[0]
print(f"""
There are {n_obs:,} observations in our state_data dataset right now. 
The unit of observation here is a (weighted) person. 
That means there's {n_obs:,} thousand rows in our dataset but they represent more than 360 thousand people. 
In fact, they represent:
""")


There are 361,475 observations in our state_data dataset right now. 
The unit of observation here is a (weighted) person. 
That means there's 361,475 thousand rows in our dataset but they represent more than 360 thousand people. 
In fact, they represent:



In [27]:
state_data['perwt'].sum()

38704629

In [28]:
n_people = state_data['perwt'].sum()
_state = state_data['stateicp'].unique()[0]
print(f"""
{n_people:,} in {_state.capitalize()}. 
Working with weighted data may be tricky for those who've never encountered it before. 
ADD MORE TEXT ABOUT WEGHTED DATA
""")



38,704,629 in California. 
Working with weighted data may be tricky for those who've never encountered it before. 
ADD MORE TEXT ABOUT WEGHTED DATA



Second step in our indexing:

In [28]:
# .isin()
mask_households_with_children = state_data['serial'].isin(households_with_children)

In [29]:
state_data[mask_households_with_children].head()

,year,serial,hhwt,stateicp,countyfip,cinethh,cihispeed,pernum,perwt,relate,sex,age,race,hispan
2185117,2017,953662,57,ohio,0,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,58,head/householder,female,48,white,not hispanic
2185118,2017,953662,57,ohio,0,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,62,child,male,20,white,not hispanic
2185119,2017,953662,57,ohio,0,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",3,78,child,female,9,white,not hispanic
2185128,2017,953668,140,ohio,61,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",1,140,head/householder,male,28,black/african american/negro,not hispanic
2185129,2017,953668,140,ohio,61,"yes, with a subscription to an internet service","yes (cable modem, fiber optic or dsl service)",2,192,sibling,female,16,black/african american/negro,not hispanic


Now we have a _working dataset_! We've taken what we **need** from the main dataset.

In [30]:
working_dataset = state_data[mask_households_with_children].copy()

Now we have a working dataset:
1. We chose one state to look at.
2. We dropped any unnecessary variables. 
3. We chose the observations we're interested in
    * Households (under a specific definition)
    * Out of those households, only those with school-age children.

No we might be tempted to continue working on this notebook but a good way to keep your analyses reproducible and organized is to separate them by tasks done. This notebook created a clean working dataset for our analysis and that's all it should do. Let's save our data and move on to the next notebook now.

We can save data in a variety of formats with pandas. 2 of the most common in social sciences are `.csv` (comma-separated values) files and `.dta` (STATA) files. 

CSV files are text files. Pandas can ___guess___ the data types of each column but this information isn't stored in the file. STATA files, in the other hand, do contain metadata. If you have categorical variables in your data, you can save those as categorical columns in your STATA file. This saves memory. However, `JupyterLab` 

ADD MORE ABOUT CSV AND STATA SAVING DATA ETC

In [31]:
working_dataset.to_stata(INTERIM_DATA_FOLDER / f'working_dataset-{today}.dta', write_index = False)